In [ ]:
# run in kernel python 3.8.10 64-bit



In [2]:
!pip install mysql-connector-python

import mysql.connector as connector

from mysql.connector.pooling import MySQLConnectionPool

     |████████████████████████████████| 593 kB 11.5 MB/s 
     |████████████████████████████████| 409 kB 110.0 MB/s 


In this exercise you must create a pool of connections and get a connection from the pool to implement the stored procedures to complete the following tasks:

* Establish a connection by importing MySQLConnectionPool and creating a pool with two connections.
* Create and call a stored procedure named PeakHours that identifies the peak, or busiest hour, for the restaurant based on the number of bookings.
* Create and call a stored procedure named GuestStatus that outputs status of each guest’s order based on which employee is assigned to the order.

-Stored procedures are created to carry out routine operations on MySQL databases. They are consistent and make sure that the written SQL queries in the procedures are executed in the same way every time you call the stored procedure.

-A stored procedure is created only once, and you store it in the MySQL database. You can call the stored procedures as many times as you need in your Python-based application.

The tasks and the steps that you must follow to complete each one are as follows:

## 1: Stablish a connection

Complete the following steps to establish a connection pool:

* To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
* To find the information on the error, import the Error class from MySQL Connector/Python.
*  Define your database configurations as a Python dictionary object called dbconfig.
* Establish a connection pool [pool_name = pool_b] with two connections. 
* Implement error handling using a try-except block in case the connection fails. 

In [3]:
dbconfig={"database":"little_lemon_db", "user":"root", "password":""}

In [4]:
## here yoyr code
try:
    pool = MySQLConnectionPool(
        pool_name="pool_b",
        pool_size=2,
        #database='little_lemon',
        #user='root',
        #password=''
        **dbconfig
        )
except Error as er:
    print("error code", er.errno)
    print("error message", er.msg)

## Task 2: 

Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.

Guest 1:

    Table Number: 8
    First Name: Anees
    Last Name: Java
    Booking Time: 18:00
    EmployeeID: 6

Guest 2:

    Table Number: 5
    First Name: Bald
    Last Name: Vin
    Booking Time: 19:00
    EmployeeID: 6

Guest 3:

    Table Number: 12
    First Name: Jay
    Last Name: Kon
    Booking Time: 19:30 
    EmployeeID: 6

Return all the connections back to the pool. 

TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.  

In [5]:
## write here your code
import datetime as dt
guests = {'g1':{'TableNo':8, 'GuestFirstName':'Anees','GuestLastName':'Java', 'BookingTime':'18:00','EmployeeID':6},
'g2':{'TableNo':5, 'GuestFirstName':'Bald','GuestLastName':'Vin', 'BookingTime':'19:00','EmployeeID':6},
'g3':{'TableNo':12, 'GuestFirstName':'Jay','GuestLastName':'Kon', 'BookingTime':'19:30','EmployeeID':6}}



In [6]:
sql_guest_insert_rows = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES (%(tableno)s, %(guestfirstname)s, %(guestlastname)s, %(bookingtime)s, %(employeeid)s ) ;
"""


for  key,value in guests.items():#range(pool.pool_size):
    print("======== Connecting for guest {} ".format(key))
    print(value)
    tableno=value['TableNo']
    guestfirstname=value['GuestFirstName']
    guestlastname=value['GuestLastName']
    bookingtime=value['BookingTime']
    employeeid=value['EmployeeID']
    try:
        conn=pool.get_connection()
    except:
        print("----full pools! adding new connections for {}: ".format(key))
        pool.add_connection()
        conn=pool.get_connection()
    ### now we have connection for sure!
    if conn.is_connected():
        cursor=conn.cursor()
        #print("the connection id for {} {} is requestinf info on booking {}".format(guests[i], conn.connection_id, i +1) )
        cursor.execute(sql_guest_insert_rows, {'tableno':tableno,'guestfirstname':guestfirstname , 'guestlastname': guestlastname,'bookingtime':bookingtime,'employeeid':employeeid})
    #    print(cursor.fetchall())
    else:
        print("no live connection made. Maybe server is down")
    # now that you inset changes you have to commit them in the database
    conn.commit()
    # now give back the connection to the pool!
    conn.close()

======== Connecting for guest g1 
{'TableNo': 8, 'GuestFirstName': 'Anees', 'GuestLastName': 'Java', 'BookingTime': '18:00', 'EmployeeID': 6}
======== Connecting for guest g2 
{'TableNo': 5, 'GuestFirstName': 'Bald', 'GuestLastName': 'Vin', 'BookingTime': '19:00', 'EmployeeID': 6}
======== Connecting for guest g3 
{'TableNo': 12, 'GuestFirstName': 'Jay', 'GuestLastName': 'Kon', 'BookingTime': '19:30', 'EmployeeID': 6}


In [7]:
### check current content in the database. 
# NOTE: because we ran twice the code, and in the first one we did not commit, the Booking ID, instead of going 7,8,9 , jumps to 10,11,12

sql_query_insert_info = """SELECT * FROM Bookings;"""

conn=pool.get_connection()
#
if conn.is_connected():
    cursor=conn.cursor()
    cursor.execute(sql_query_insert_info)
    results_test = cursor.fetchall()
    print(cursor.column_names)
    for row in results_test:
        print(row)
else:
    print("no live connection made")
conn.close()

('BookingID', 'TableNo', 'GuestFirstName', 'GuestLastName', 'BookingSlot', 'EmployeeID')
(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1)
(2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1)
(3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3)
(4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4)
(5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2)
(6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)
(10, 8, 'Anees', 'Java', datetime.timedelta(seconds=64800), 6)
(11, 5, 'Bald', 'Vin', datetime.timedelta(seconds=68400), 6)
(12, 12, 'Jay', 'Kon', datetime.timedelta(seconds=70200), 6)


## Task 3: 

Create a report containing the following information:

1.  The name and EmployeeID of the Little Lemon manager
2.  The name and role of the employee who receives the highest salary.
3.  The number of guests booked between 18:00 and 20:00.
4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [47]:
## your code here
sql_report_pseudocode ="""
SELECT EmployeeID of manager,
    Name and Role of employee with highest salry,
    n_guests booked between 18 and 20
    full name and bookingid of guests waiting to be seated (receptionists in sorted order respect booking slot)
"""


sql_report_1 ="""
SELECT EmployeeID,
    Name,
    Role
    FROM Employees
    WHERE Role ='Manager';
"""
sql_report_2 ="""
SELECT EmployeeID,
    Name,
    Role,
    CAST(REPLACE(REPLACE(Annual_Salary,"$",""),",","") AS UNSIGNED INTEGER) AS annualsalary
FROM Employees
ORDER BY annualsalary DESC
LIMIT 1;
"""
sql_report_3 ="""
SELECT BookingID,
    HOUR(BookingSlot) as hourbooking
FROM Bookings
WHERE HOUR(BookingSlot) >=18 
    AND  HOUR(BookingSlot) <=20 
;
"""

sql_report_4 ="""
SELECT  BookingID,
    CONCAT(GuestFirstName,' ',GuestLastName) as fullname
    FROM Bookings
    ORDER BY BookingSlot ;
"""





In [65]:
#
if conn.is_connected():
    conn.close()

In [60]:
### performs all the calls to the sqls in order and return the values into a variable before print all the results in a report style:

conn=pool.get_connection()
#
if conn.is_connected():
    cursor=conn.cursor(buffered=True)
    cursor.execute(sql_report_1)
    #1.  The name and EmployeeID of the Little Lemon manager
    #2.  The name and role of the employee who receives the highest salary.
    #3.  The number of guests booked between 18:00 and 20:00.
    #4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
    print("================================================================================") 
    print("=================================== REPORT =====================================")
    print("================================================================================\n") 
    print("-----------------------------------------------------------------------------------")
    print("1.  The name and EmployeeID of the Little Lemon manager\n")
    print(cursor.column_names)
    results_1 = cursor.fetchall()
    for r1 in results_1:
        print(r1) #cursor.column_names)
    print("-----------------------------------------------------------------------------------")
    #for row in results_test:
    #    print(row)
    cursor.execute(sql_report_2)
    print("-----------------------------------------------------------------------------------")
    print("2.  The name and role of the employee who receives the highest salary.\n")
    print(cursor.column_names)
    results_2 = cursor.fetchall()
    for r2 in results_2:
        print(r2)
    print("-----------------------------------------------------------------------------------")
    #
    cursor.execute(sql_report_3)
    print("-----------------------------------------------------------------------------------")
    print("3.  The number of guests booked between 18:00 and 20:00.\n")
    print(cursor.column_names)
    results_3 = cursor.fetchall()
    for r3 in results_3:
        print(r3)
    print("-----------------------------------------------------------------------------------")
    #
    cursor.execute(sql_report_4)
    print("-----------------------------------------------------------------------------------")
    print("4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.\n")
    print(cursor.column_names)
    results_4 = cursor.fetchall()
    for r4 in results_4:
        print(r4)
    print("-----------------------------------------------------------------------------------")
    #
    print("================================================= END OF REPORT ===========================================")
    # 

else:
    print("no live connection made")
conn.close()


=================================== REPORT =====================================

-----------------------------------------------------------------------------------
1.  The name and EmployeeID of the Little Lemon manager

('EmployeeID', 'Name', 'Role')
(1, 'Mario Gollini', 'Manager')
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
2.  The name and role of the employee who receives the highest salary.

('EmployeeID', 'Name', 'Role', 'annualsalary')
(1, 'Mario Gollini', 'Manager', 70000)
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
3.  The number of guests booked between 18:00 and 20:00.

('BookingID', 'hourbooking')
(1, 19)
(2, 19)
(5, 18)
(6, 20)
(10, 18)
(11, 19)
(12, 19)
--------------------------------------------------------------------------

## TASK 4:

Create a stored procedure named BasicSalesReport that returns the following statistics: 

* Total sales
* Average sale
* Minimum bill paid
* Maximum bill paid

In [63]:
sql_procedure_basicsales = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT SUM(BillAmount) AS 'Total sales',
        AVG(BillAmount) AS 'Average sale',
        MIN(BillAmount) AS 'Minimum bill paid',
        MAX(BillAmount) AS 'Maximum bill paid'
    FROM Orders;
END
"""

In [66]:
# create and execute
# STEP 2
conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor()
    cursor.execute(sql_procedure_basicsales)
    print("procedure basicsales created")
else:
    print("no live connection made")
conn.close()
print("the connection is placed back into the pool for the next user to connect")



procedure basicsales created
the connection is placed back into the pool for the next user to connect


In [67]:
# STEP 3 : call procedure

conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor()
    cursor.callproc("BasicSalesReport")
    
    #dataset = cursor.fetchall() NOT for PROCEDURES
    results = next(cursor.stored_results())
    #print(results.description)
    columns = results.column_names
    print(columns)
    dataset = results.fetchall()

    for row in dataset:
        print(row)
else:
    print("no live connection made")
conn.close()
print("the connection is placed back into the pool for the next user to connect")

('Total sales', 'Average sale', 'Minimum bill paid', 'Maximum bill paid')
(Decimal('243'), Decimal('48.6000'), 37, 86)
the connection is placed back into the pool for the next user to connect


## TASK 5: 
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

* Get a connection from the pool.
* Create a buffered cursor.
* Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 
* Returned the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [71]:
## SQL test
sql_test_chefnotifications = """ 
SELECT 
    Bookings.BookingID,
    Bookings.BookingSlot,
    CONCAT(Bookings.GuestFirstName,' ',Bookings.GuestLastName) AS Guest_name,
    Employees.Name AS EmployeeName,
    Employees.Role
FROM Bookings
LEFT JOIN Employees
ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY BookingSlot ASC
LIMIT 3;
"""



In [73]:
if conn.is_connected():
    conn.close()

In [76]:
import datetime as dt

## check
conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor(buffered=True)
    cursor.execute(sql_test_chefnotifications)
    notificationinfo=cursor.fetchall()
    print(notificationinfo)
    #print("procedure basicsales created")
    for notification in notificationinfo:
        print("-----------------------\n Booking slot {} .\n Guest name: {} .\n Assigned to {} : ({}) .\n-----------------------".format(notification[1] ,notification[2], notification[3], notification[4] ))
else:
    print("No live connection made")
conn.close()
print("\n\nThe connection is placed back into the pool for the next user to connect.")

[(3, datetime.timedelta(seconds=54000), 'Vanessa McCarthy', 'Giorgos Dioudis', 'Head Chef'), (4, datetime.timedelta(seconds=63000), 'Marcos Romero', 'Fatma Kaya', 'Assistant Chef'), (10, datetime.timedelta(seconds=64800), 'Anees Java', 'John Millar', 'Receptionist')]
-----------------------
 Booking slot 15:00:00 .
 Guest name: Vanessa McCarthy .
 Assigned to Giorgos Dioudis : (Head Chef) .
-----------------------
-----------------------
 Booking slot 17:30:00 .
 Guest name: Marcos Romero .
 Assigned to Fatma Kaya : (Assistant Chef) .
-----------------------
-----------------------
 Booking slot 18:00:00 .
 Guest name: Anees Java .
 Assigned to John Millar : (Receptionist) .
-----------------------


the connection is placed back into the pool for the next user to connect
